# Primeros pasos con pandas en Snowflake

[pandas on Snowflake](https://docs.snowflake.com/developer-guide/snowpark/python/snowpark-pandas) permite a los desarrolladores ejecutar su código de pandas directamente en sus datos en Snowflake. Los usuarios podrán obtener la misma experiencia nativa de pandas que conocen y aman con el rendimiento, la escala y la gobernanza de Snowflake.

En esta guía de inicio rápido, mostraremos cómo puede comenzar a ejecutar pandas en Snowflake a través de la API de pandas de Snowpark. También veremos que la API de pandas de Snowpark es muy similar a la API nativa de pandas y le permite escalar sus canalizaciones tradicionales de pandas con solo unas pocas líneas de cambio. Puede ejecutar este cuaderno en un cuaderno de Snowflake.

## Uso de la API de pandas de Snowpark

La API de pandas de Snowpark está disponible como parte del paquete Snowpark Python (versión 1.17 y superior). Snowpark Python viene preinstalado con el entorno de Snowflake Notebooks. Además, deberá agregar el paquete `modin` en el menú desplegable `Packages`.

- Para instalar Modin, seleccione `modin` de `Packages` y asegúrese de que la versión sea `0.30.1`.

In [ ]:
# Import the Snowpark pandas plugin for modin
import snowflake.snowpark.modin.plugin
import modin.pandas as pd

## Crear sesión de Snowpark
Snowpark pandas requiere un objeto `Session` activo para conectarse a sus datos en Snowflake. En la siguiente celda, inicializaremos un objeto Session e importaremos Snowpark pandas como `pd`. Asegúrese de utilizar una base de datos en la que tenga permisos de escritura al crear la sesión, ya que Snowpark pandas requiere permisos de escritura.

In [ ]:
# Access current Snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()
# Add a query tag to the session for troubleshooting and monitoring
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"pandas_on_snowflake", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook", "vignette":"snowpark_pandas"}}

## Lectura de datos desde Snowflake
Hoy, analizaremos los datos de series de tiempo del [conjunto de datos de Finanzas y Economía de Cybersyn](https://app.snowflake.com/marketplace/listing/GZTSZAS2KF7/cybersyn-inc-financial-economic-essentials). Puede encontrar las instrucciones para configurar el conjunto de datos para este tutorial [aquí](https://quickstarts.snowflake.com/guide/getting_started_with_pandas_on_snowflake/#1).

¡Comencemos leyendo la tabla `stock_price_timeseries` en un DataFrame!

Verifique que tengas permisos de escritura en la base de datos con la que inicializó la `Session` de Snowpark. Si está leyendo de la tabla `stock_price_timeseries`, su `Session` debe configurarse para usar una base de datos diferente en la que tenga permisos de escritura. La celda a continuación utiliza el nombre completo de la tabla para garantizar que la lectura se realice correctamente, incluso si la `Session` está configurada para usar una base de datos diferente.

In [ ]:
-- Cuantos registros tiene la tabla?
select count(1) from PANDAS_DB.PUBLIC.STOCK_PRICE_TIMESERIES;

In [ ]:
# Read data into a Snowpark pandas df 
from time import perf_counter
start = perf_counter()
spd_df = pd.read_snowflake("PANDAS_DB.PUBLIC.STOCK_PRICE_TIMESERIES")
end = perf_counter()
data_size = len(spd_df)
print(f"Snowpark pandas tardó {round(end - start,3)} segundos en leer una tabla con {data_size:,} registros!")
snow_time = end - start

Ahora hagamos lo mismo leyendo los datos en pandas nativo. Hay dos enfoques comunes para hacer esto:

1) Crear un [DataFrame de Snowpark](https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes#return-the-contents-of-a-dataframe-as-a-pandas-dataframe) y llamar a [`to_pandas`](https://docs.snowflake.com/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.to_pandas) para exportar los resultados a un DataFrame de pandas.
```python
snowpark_df = session.table("STOCK_PRICE_TIMESERIES")
native_pd_df = snowpark_df.to_pandas()
```

2) Usemos el [Snowflake Connector for Python](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-pandas) para consultar y exportar los resultados desde Snowflake a un Dataframe de Pandas usando [`fetch_pandas_all`](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-api#fetch_pandas_all)

```python
# Crear un objeto cursor
cur = session.connection.cursor()
# Ejecutar una sentencia que generará un conjunto de resultados
cur.execute("select * from STOCK_PRICE_TIMESERIES")
# Obtener todas las filas de un cursor y cargarlas en un DataFrame de pandas
native_pd_df = cur.fetch_pandas_all()
```

Utilizaremos el segundo enfoque a continuación y mediremos el tiempo que tardan estas operaciones. (Nota: ¡Esto puede tardar varios minutos!)

In [ ]:
start = perf_counter()
cur = session.connection.cursor()
cur.execute("select * from PANDAS_DB.PUBLIC.STOCK_PRICE_TIMESERIES")
native_pd_df = cur.fetch_pandas_all()
end = perf_counter()
print(f"Pandas nativo tardó {round(end - start,3)} segundos en leer la misma tabla con {data_size:,} registros!!")

Como puede ver, lleva mucho más tiempo exportar la tabla de Snowflake a la memoria para operar con pandas nativo que para que Snowpark pandas lea la tabla directamente. Esto también puede provocar que la sesión del cuaderno se bloquee si los datos exportados exceden lo que cabe en la memoria.

## Examina los Datos Crudos
Echemos un vistazo a los datos con los que vamos a trabajar. Inspeccionaremos las primeras cinco filas del dataframe y las imprimiremos utilizando la visualización interactiva de dataframes de Streamlit (`st.dataframe`).

In [ ]:
import streamlit as st
st.dataframe(spd_df.head(5))

## Filtrando los Datos
Echemos un vistazo a algunas transformaciones de datos comunes, ¡empezando por el filtrado! ¡Filtremos las acciones que cotizan en la Bolsa de Nueva York!

In [ ]:
start = perf_counter()
nyse_spd_df = spd_df[(spd_df['PRIMARY_EXCHANGE_CODE'] == 'NYS')]
repr(nyse_spd_df)
end = perf_counter()
st.dataframe(nyse_spd_df.head())
print(f"Filtrando por acciones pertenecientes a la NYSE tardó {round(end - start,3)} segundos en Snowpark pandas")

## Filtrado Granular de Datos
Intentemos un filtro aún más granular: ¡filtremos para la apertura pre-mercado de las acciones que tienen los siguientes tickers:
* GOOG (Alphabet, Inc.)
* MSFT (Microsoft)
* SNOW (Snowflake)

In [ ]:
start = perf_counter()
filtered_spd_df = spd_df[((spd_df['TICKER'] == 'GOOG') | (spd_df['TICKER'] == 'MSFT') | (spd_df['TICKER'] == 'SNOW')) & (spd_df['VARIABLE_NAME'] == 'Pre-Market Open')]
repr(filtered_spd_df)
end = perf_counter()
st.dataframe(filtered_spd_df.head())
print(f"Filtrar por el precio de apertura pre-mercado para las acciones mencionadas anteriormente tardó {round(end - start,3)} segundos en Snowpark pandas")

# Transformando los Datos
Digamos que quisiéramos analizar el rendimiento de varios precios de acciones a lo largo del tiempo; en ese caso, podría ser más útil tener los valores como columnas, y el nombre del ticker y la fecha como índice, en lugar de la codificación actual. ¡Podemos lograr esto utilizando la API `pivot_table`!

In [ ]:
start = perf_counter()
reshape_df = spd_df.pivot_table(index=["TICKER", "DATE"], columns="VARIABLE_NAME", values="VALUE")
repr(reshape_df)
end = perf_counter()
print(f"Pivotar el DataFrame tardó {round(end - start,3)} segundos en Snowpark pandas")

In [ ]:
st.dataframe(reshape_df.head())

## Transformando los Datos
Ahora que hemos reformateado los datos, podemos comenzar a aplicar algunas transformaciones. Empecemos por echar un vistazo a la columna de Mínimo Diario para los tickers mencionados anteriormente. ¡Podemos remuestrear los datos para observar el Mínimo Trimestral para el ticker `GOOG`!

In [ ]:
start = perf_counter()
resampled_spd_df_all_quarter_low = reshape_df["All-Day Low"]["GOOG"].resample("91D").min()
repr(resampled_spd_df_all_quarter_low)
end = perf_counter()
print(f"Remuestrear el DataFrame tardó {round(end - start,3)} segundos en Snowpark pandas")

In [ ]:
print(resampled_spd_df_all_quarter_low)

Incluso podemos observar la fluctuación de precios trimestre a trimestre utilizando la API `diff`!

In [ ]:
start = perf_counter()
q_o_q_resampled_spd_df_all_quarter_low = resampled_spd_df_all_quarter_low.diff()
repr(q_o_q_resampled_spd_df_all_quarter_low)
end = perf_counter()
print(f"Calcular las diferencias de los datos remuestreados tardó {round(end - start,3)} segundos en Snowpark pandas")

In [ ]:
print(q_o_q_resampled_spd_df_all_quarter_low)

## Aplicar una función a lo largo de un eje
Ahora queremos aplicar la raíz cuadrada del valor absoluto a cada valor de la serie.

Snowpark pandas soporta `apply`, que aplica alguna función arbitraria de Python definida por el usuario a lo largo de un eje particular del DataFrame o Serie.

La función de Python se serializa en bytecode de Python y se ejecuta como una UDF dentro del entorno de ejecución del sandbox seguro de Python de Snowpark. El entorno de ejecución de Python de Snowpark está perfectamente integrado con el administrador de paquetes Anaconda, de modo que los usuarios pueden aprovechar sus paquetes de terceros favoritos, como NumPy, para una transformación de datos flexible dentro de su `dataframe.apply`.

**Consejo de Experto:** Si bien llamar a `apply` es conveniente, dado que la implementación subyacente son UDF o UDTF, puede que no esté tan optimizado como las consultas SQL transpiled de otras consultas de Snowpark pandas. Si la función aplicada tiene una operación de dataframe o serie equivalente, recomendamos utilizar esas operaciones en su lugar. Por ejemplo, en lugar de `df.groupby('col1').apply('sum')`, llama directamente a `df.groupby('col1').sum()`.

In [ ]:
import numpy as np
resampled_all_quarter_low_df_sqrt = q_o_q_resampled_spd_df_all_quarter_low.apply(
    lambda x: np.sqrt(abs(x))
)

In [ ]:
resampled_all_quarter_low_df_sqrt = resampled_all_quarter_low_df_sqrt.dropna()
print(resampled_all_quarter_low_df_sqrt)

## Visualizando tus resultados con Altair

pandas se utiliza a menudo en conjunto con bibliotecas de visualización y aprendizaje automático de terceros. Aquí queremos trazar la fluctuación de precios trimestre a trimestre como un gráfico de barras.

Primero, limpiemos los datos para fines de trazado.

In [ ]:
# Convert series to dataframe by resetting index
plot_df = q_o_q_resampled_spd_df_all_quarter_low.reset_index()
# Rename columns
plot_df.columns = ["DATE", "QLOW_DIFF"]
# Filter out extreme values
plot_df = plot_df[plot_df["QLOW_DIFF"]>-700]

Al llamar a APIs de bibliotecas de terceros con un dataframe de Snowpark pandas, recomendamos convertir el dataframe de Snowpark pandas a un dataframe de pandas llamando a [`to_pandas`](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.21.0/modin/pandas_api/snowflake.snowpark.modin.pandas.to_pandas) antes de pasar el dataframe a la llamada de la biblioteca de terceros.

Ten en cuenta que llamar a `to_pandas` extrae tus datos de Snowflake y los carga en la memoria, así que procede con precaución para conjuntos de datos grandes y casos de uso sensibles. Generalmente recomendamos agregar o resumir y exportar solo los datos que utilizarás para trazar utilizando `to_pandas`.


In [ ]:
print("Tipo de DataFrame de entrada: ", type(plot_df))
pandas_plot_df = plot_df.to_pandas()
print("Después de to_pandas, tipo de DataFrame de salida: ", type(pandas_plot_df))

Ahora podemos usar cualquier biblioteca de visualización de Python, como Altair, para trazar el dataframe de pandas resultante.

In [ ]:
import altair as alt
alt.Chart(pandas_plot_df).mark_bar(width=10).encode(
    x = alt.X("DATE:T"),
    y = alt.Y("QLOW_DIFF:Q"),
    color=alt.condition(
        alt.datum.QLOW_DIFF > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
)

### Conclusión
pandas en Snowflake libera el poder de Snowflake para los desarrolladores de pandas al permitirte ejecutar la misma API de pandas, mientras operas con grandes conjuntos de datos que típicamente no funcionan con pandas nativo y, ¡todo ello manteniendo tus datos en Snowflake! Para obtener más información, consulta la [Documentación de Snowflake](https://docs.snowflake.com/developer-guide/snowpark/python/snowpark-pandas). Para un ejemplo más avanzado, consulta [este inicio rápido](https://quickstarts.snowflake.com/guide/data_engineering_pipelines_with_snowpark_pandas/) sobre cómo puedes construir un pipeline de ingeniería de datos con Snowpark pandas.